In [1]:
%cd ..

/home/bhkuser/bhklab/katy/readii_2_roqc


In [2]:
from pathlib import Path

import numpy as np
import pandas as pd
import yaml
from damply import dirs
from readii.io.loaders import loadFileToDataFrame, loadImageDatasetConfig
from readii.process.label import (
    eventOutcomeColumnSetup,
    getPatientIdentifierLabel,
    timeOutcomeColumnSetup,
)
from readii.process.split import splitDataByColumnValue
from readii.process.subset import getPatientIntersectionDataframes
from sklearn.linear_model import Lasso
from sksurv.metrics import concordance_index_censored

In [3]:
dirs

DamplyDirs<Structure: NESTED>
Project Root: /home/bhkuser/bhklab/katy/readii_2_roqc
CONFIG       : ├── config
LOGS         : ├── logs
METADATA     : ├── metadata
NOTEBOOKS    : ├── workflow/notebooks
PROCDATA     : ├── data/procdata
RAWDATA      : ├── data/rawdata
RESULTS      : ├── data/results
SCRIPTS      : └── workflow/scripts

### Set up variables from config file for the dataset

In [8]:
# Load in the configuration file
config = loadImageDatasetConfig("RADCURE", dirs.CONFIG / "datasets")

# Initialize dataset parameters
CLINICAL_DATA_FILE = config["CLINICAL"]["FILE"]
OUTCOME_VARIABLES = config["CLINICAL"]["OUTCOME_VARIABLES"]

DATASET_NAME = config["DATA_SOURCE"] + "_" + config["DATASET_NAME"]

RANDOM_SEED = config['RANDOM_SEED']

# train test split settings
TRAIN_TEST_SPLIT = config["ANALYSIS"]["TRAIN_TEST_SPLIT"]

pyradiomics_settings = config["EXTRACTION"]["CONFIG"]

signature_name = "choi_opc_hpv_2020"
signature_file = dirs.CONFIG / "signatures" / signature_name

prediction_dir = dirs.RESULTS / DATASET_NAME / "prediction" / signature_name
prediction_dir.mkdir(parents=True, exist_ok=True)

# Intersect Clinical and Feature data to get patient subset for analysis

## Clinical data loading and processing

In [ ]:
# Load clinical data file
raw_clinical_data = loadFileToDataFrame((dirs.PROCDATA / DATASET_NAME / "clinical" / "OPC_only_RADCURE_Clinical_20250731.csv"))
clinical_pat_id = getPatientIdentifierLabel(raw_clinical_data)

if TRAIN_TEST_SPLIT['split']:
    split_data = splitDataByColumnValue(raw_clinical_data,
                                        split_col_data=TRAIN_TEST_SPLIT['split_variable'],
                                        impute_value=TRAIN_TEST_SPLIT['impute'])
    # TODO: don't default to test string
    clinical_data = split_data['test']

# Load the extraction index to use for mapping TCIA IDs to local file names
mit_index = loadFileToDataFrame(dirs.PROCDATA / DATASET_NAME / "features" / )

if 'SampleID' not in mit_index.columns:
    mit_index['SampleID'] = config["DATASET_NAME"] + "_" + mit_index['SampleNumber'].astype(str).str.zfill(3)

# SampleID is local file name
# PatientID is TCIA ID
id_map = mit_index['SampleID']
id_map.index = mit_index["PatientID"]
id_map = id_map.drop_duplicates()

# Map the SampleIDs to the clinical data and add as a column for intersection
raw_clinical_data['SampleID'] = raw_clinical_data[clinical_pat_id].map(id_map)
raw_clinical_data = raw_clinical_data.set_index('SampleID')

KeyError: 'MIT_INDEX_FILE'

## Feature data loading and processing

In [6]:
image_type = "full_original"
raw_feature_data = loadFileToDataFrame((dirs.RESULTS / pyradiomics_settings / f"{image_type}_features.csv"))

raw_feature_data.rename(columns={"ID": "SampleID"}, inplace=True)
# Set the index to SampleID
raw_feature_data.set_index('SampleID', inplace=True)

12:24:00 [error    ] File /Users/katyscott/Documents/BHKLab_GitHub/readii_2_roqc/data/results/pyradiomics_original_single_feature.yaml/full_original_features.csv does not exist [readii] call=general.loadFileToDataFrame:95


FileNotFoundError: 

## Intersect clinical and feature data

In [23]:
clinical_data, pyrad_subset = getPatientIntersectionDataframes(raw_clinical_data, raw_feature_data, need_pat_index_A=False, need_pat_index_B=False)

# Set up outcome columns in clinical data

In [24]:
clinical_data = eventOutcomeColumnSetup(dataframe_with_outcome=clinical_data,
                                        outcome_column_label=OUTCOME_VARIABLES["event_label"],
                                        standard_column_label="survival_event_binary",
                                        )
clinical_data = timeOutcomeColumnSetup(dataframe_with_outcome=clinical_data,
                                       outcome_column_label=OUTCOME_VARIABLES["time_label"],
                                       standard_column_label="survival_time_years",
                                       convert_to_years=OUTCOME_VARIABLES["convert_to_years"])

# Feature Selection

## Existing signature

### Load signature from yaml file

In [ ]:
# Path to the yaml file in the rawdata directory
radiomic_signature_yaml = dirs.RAWDATA.parent / "cph_weights_radiomic_signature" / "aerts_original.yaml"

try:
    with open(radiomic_signature_yaml, 'r') as f:
        yaml_data = yaml.safe_load(f)
        if not isinstance(yaml_data, dict):
            raise TypeError("ROI match YAML must contain a dictionary")
        radiomic_signature = pd.Series(yaml_data['signature'])
except Exception as e:
    print(f"Error loading YAML file: {e}")
    raise

### Get just features in radiomic signature

In [29]:
signature_feature_data = pyrad_subset[radiomic_signature.index]

### Add outcome columns, and save out for R processing

In [11]:
signature_plus_outcomes = pd.concat([clinical_data[['survival_event_binary', 'survival_time_years']], signature_feature_data], axis=1)


signature_plus_outcomes.to_csv(signature_dir / f"{image_type}.csv", index=True)

## LASSO feature selection

In [ ]:
def calc_LASSO(X_train : pd.DataFrame, 
               event_status_train : pd.DataFrame, 
               a_lasso : float
               ) -> np.array:
    """
    Performs LASSO for feature selection
   
    Parameters
    -----------
    X_train: pd.DataFrame
        A dataframe containing only radiomic features
    event_status_train: pd.DataFrame
        A column containing the event status of patients
    a_lasso: float
        The regularization parameter alpha to be used when initializing the LASSO model
       
    Returns
    ----------
    lasso_passed: list
        All feature names with non zero coefficients
    """
 
    lasso = Lasso(a_lasso)
    lasso.fit(X_train, event_status_train)
    lasso_coef = np.abs(lasso.coef_)
 
    all_rad_feats = X_train.columns.to_numpy().tolist()
    lasso_passed = np.array(all_rad_feats)[lasso_coef>0]
 
    return(lasso_passed)

In [ ]:
from readii.process import getOnlyPyradiomicsFeatures

feats_only = getOnlyPyradiomicsFeatures(pyrad_subset)
event_status_train = clinical_data['survival_event_binary']
calc_LASSO(feats_only, event_status_train, 0.5)

# Prediction Modelling

## With existing signature weights

In [30]:
feature_hazards = signature_feature_data.dot(radiomic_signature)

In [31]:
cindex, concordant, discordant, _tied_risk, _tied_time = concordance_index_censored(
    event_indicator = clinical_data['survival_event_binary'].astype(bool),
    event_time = clinical_data['survival_time_years'],
    estimate = feature_hazards,
    )

### Bootstrap to get confidence intervals

In [ ]:
sampled_cindex_bootstrap = []

hazards_and_outcomes = pd.DataFrame({
    'hazards': feature_hazards,
    'survival_event_binary': clinical_data['survival_event_binary'],
    'survival_time_years': clinical_data['survival_time_years']
}, index=clinical_data.index)

bootstrap_count = 1000

for _idx in range(bootstrap_count):
    sampled_results = hazards_and_outcomes.sample(n=hazards_and_outcomes.shape[0], replace=True)

    sampled_cindex, _concordant, _discordant, _tied_risk, _tied_time = concordance_index_censored(
        event_indicator = sampled_results['survival_event_binary'].astype(bool),
        event_time = sampled_results['survival_time_years'],
        estimate = sampled_results['hazards'],
        )
    
    sampled_cindex_bootstrap.append(sampled_cindex)

lower_confidence_interval = sorted(sampled_cindex_bootstrap)[bootstrap_count // 4 - 1]
upper_confidence_interval = sorted(sampled_cindex_bootstrap)[bootstrap_count - (bootstrap_count // 4)]

In [ ]:
# print("Results")
# print("C-index: ", cindex)
# print("Lower confidence interval: ", lower_confidence_interval)
# print("Upper confidence interval: ", upper_confidence_interval)

Results
C-index:  0.4914219249781913
Lower confidence interval:  0.4663610823598994
Upper confidence interval:  0.5148074267300905
